Readme: 
1.The first part shows the procedure of user_based CF and I give one example at the end of this part
2.The second part shows the procedure of item_based CF and I give one example at the end of this part

In [2]:
import numpy as np

In [122]:
#there are 943 users and 1682 items
user_item_matrix=np.full((943+1,1682+1),-1) #each row represents a user and each col represents a item, user_item_matrix[i][j]=user i's rating of item j
file = open('ml-100k/u.data', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('\t')
    user_id=int(splits[0])
    item_id=int(splits[1])
    rating=int(splits[2])

    user_item_matrix[user_id][item_id]=rating

1.User-based Collaborative Filtering

In [39]:
def sort_dict(dict):
    dict=sorted(dict.items(), key=lambda x:x[1],reverse=True)   
    return dict

In [96]:
def calculate_user_AverageRatings(user_item_matrix):
    user_avg_rating={}
    row=len(user_item_matrix)
    col=len(user_item_matrix[0])
    for i in range(1,row):
        total=0
        cnt=0
        for j in range(1,col):
            if user_item_matrix[i][j]!=-1:
                total+=user_item_matrix[i][j]
                cnt+=1
        user_avg_rating[i]=total/cnt
    return user_avg_rating



In [139]:
def calculate_user_Similarity(user_item_matrix,user_id,item_id):
    row=len(user_item_matrix)
    col=len(user_item_matrix[0])
    sim_dict={}
    compare_index=[] # col that will be used to calculate similarity

    for j in range(1,col):
        if j!=item_id and user_item_matrix[user_id][j]!=-1:
            compare_index.append(j)

    for i in range(1,row):
        if i==user_id:
            continue
        
        sum=0
        sqrt_1=0
        sqrt_2=0
        for j in compare_index:
            if user_item_matrix[i][j]!=-1:
                sum+=user_item_matrix[i][j]*user_item_matrix[user_id][j]
                sqrt_1+=user_item_matrix[i][j]**2
                sqrt_2+=user_item_matrix[user_id][j]**2
        if sqrt_1*sqrt_2!=0:
            sim_score=sum/np.sqrt(sqrt_1*sqrt_2)
            sim_dict[i]=sim_score
        else:
            sim_dict[i]=0

    dict=sort_dict(sim_dict)
    sorted_sim_dictionary={}

    for each in dict:
        sorted_sim_dictionary[each[0]]=each[1]

    return sorted_sim_dictionary

        


In [140]:
def get_TopK_neighbor_index(sorted_sim_dict,neighbor_size):
    cnt=1
    top_k_neighbor_index=[]
    for each in sorted_sim_dict:
        if cnt>neighbor_size:
            break
        cnt+=1      
        top_k_neighbor_index.append(each)
    return top_k_neighbor_index


In [141]:
def user_based_CF(user_item_matrix,neighbor_size,user_id,item_id):
    user_avg_rating_dict=calculateAverageRatings(user_item_matrix)
    sorted_sim_dict=calculate_user_Similarity(user_item_matrix,user_id,item_id)
    top_k_neighbor_index=get_TopK_neighbor_index(sorted_sim_dict,neighbor_size)

    rating=user_avg_rating_dict[user_id]
    sim_sum=0
    for index in top_k_neighbor_index:
        sim_sum+=sorted_sim_dict[index]
    
    for index in top_k_neighbor_index:
        if user_item_matrix[user_id][item_id]!=-1:
            score=sorted_sim_dict[index]*(user_item_matrix[user_id][item_id]-user_avg_rating_dict[index])/sim_sum
            rating+=score
    print("rating of user_id="+str(user_id)+" to item_id="+str(item_id)+" is "+str(rating))
    return rating




In [142]:
neighbor_size=5
user_id=1
item_id=2
rating=user_based_CF(user_item_matrix,neighbor_size,user_id,item_id)


rating of user_id=1 to item_id=2 is 3.330640437993379


In [130]:
print("From user_item_matrix provided by the website, rating of user_id="+str(user_id)+" to item_id="+str(item_id)+" is "+str(user_item_matrix[user_id][item_id]))
print("We can see that the predicted value of my user-based CF is close to actual value")

From user_item_matrix provided by the website, rating of user_id=1 to item_id=2 is 3
We can see that the predicted value of my user-based CF is close to actual value


2.Item Based Collaborative Filtering

In [120]:
#there are 943 users and 1682 items
item_user_matrix=np.full((1682+1,943+1),-1) #each row represents a user and each col represents a item, user_item_matrix[i][j]=user i's rating of item j
file = open('ml-100k/u.data', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('\t')
    user_id=int(splits[0])
    item_id=int(splits[1])
    rating=int(splits[2])

    item_user_matrix[item_id][user_id]=rating

In [101]:
def calculate_item_AverageRatings(item_user_matrix):
    item_avg_rating={}
    row=len(item_user_matrix)
    col=len(item_user_matrix[0])
    for i in range(1,row):
        total=0
        cnt=0
        for j in range(1,col):
            if item_user_matrix[i][j]!=-1:
                total+=item_user_matrix[i][j]
                cnt+=1
        item_avg_rating[i]=total/cnt
    return item_avg_rating

In [134]:
def calculate_item_Similarity(item_user_matrix,user_id,item_id):
    row=len(item_user_matrix)
    col=len(item_user_matrix[0])
    sim_dict={}
    compare_index=[] # col that will be used to calculate similarity

    for j in range(1,col):
        if j!=user_id and item_user_matrix[item_id][j]!=-1:
            compare_index.append(j)

    for i in range(1,row):
        if i==item_id:
            continue
        
        sum=0
        sqrt_1=0
        sqrt_2=0
        for j in compare_index:
            if item_user_matrix[i][j]!=-1:
                sum+=item_user_matrix[i][j]*item_user_matrix[item_id][j]
                sqrt_1+=item_user_matrix[i][j]**2
                sqrt_2+=item_user_matrix[item_id][j]**2
        
        if sqrt_1*sqrt_2!=0:
            sim_score=sum/np.sqrt(sqrt_1*sqrt_2)
            sim_dict[i]=sim_score
        else:
            sim_dict[i]=0

    dict=sort_dict(sim_dict)
    sorted_sim_dictionary={}

    for each in dict:
        sorted_sim_dictionary[each[0]]=each[1]

    return sorted_sim_dictionary

In [118]:
def item_based_CF(item_user_matrix,neighbor_size,user_id,item_id):
    item_avg_rating_dict=calculateAverageRatings(item_user_matrix)
    sorted_sim_dict=calculate_item_Similarity(item_user_matrix,user_id,item_id)
    top_k_neighbor_index=get_TopK_neighbor_index(sorted_sim_dict,neighbor_size)

    rating=item_avg_rating_dict[item_id]
    sim_sum=0
    for index in top_k_neighbor_index:
        sim_sum+=sorted_sim_dict[index]
    
    for index in top_k_neighbor_index:
        if item_user_matrix[item_id][user_id]!=-1:
            score=sorted_sim_dict[index]*(item_user_matrix[item_id][user_id]-item_avg_rating_dict[index])/sim_sum
            rating+=score
    print("rating of user_id="+str(user_id)+" to item_id="+str(item_id)+" is "+str(rating))
    return rating

In [135]:
neighbor_size=5
user_id=1
item_id=2
rating=item_based_CF(item_user_matrix,neighbor_size,user_id,item_id)

rating of user_id=1 to item_id=2 is 2.9372179813401185


In [136]:
print("From user_item_matrix provided by the website, rating of user_id="+str(user_id)+" to item_id="+str(item_id)+" is "+str(user_item_matrix[user_id][item_id]))
print("We can see that the predicted value of my user-based CF is close to actual value")

From user_item_matrix provided by the website, rating of user_id=1 to item_id=2 is 3
We can see that the predicted value of my user-based CF is close to actual value


3.Summary

In [143]:
# to implement user_based_CF
neighbor_size=83
user_id=14
item_id=2
rating=user_based_CF(user_item_matrix,neighbor_size,user_id,item_id)

rating of user_id=14 to item_id=2 is 4.091836734693878


In [138]:
#to implement item_based CF
neighbor_size=83
user_id=14
item_id=2
rating=item_based_CF(item_user_matrix,neighbor_size,user_id,item_id)

rating of user_id=14 to item_id=2 is 3.2061068702290076


In [144]:
print(item_user_matrix[user_id][item_id])

4
